In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))


Ready to use Azure ML 1.31.0 to work with ws_Machine_Learning


In [ ]:
from azureml.core import Experiment, Run
import pandas as pd

"""
Available experiments:
- exp-2-PyTorch-2D-CNN-train_noCV
- exp-2-PyTorch-2D-CNN-train_CV
- exp-3-PyTorch-2D-CNN-train_CV
- exp-3-PyTorch-2D-CNN-train_noCV
- exp-4-PyTorch-2D-CNN-train_CV
- exp-4-PyTorch-2D-CNN-train_noCV
"""
# Load experiment
experiment = Experiment(ws, name='exp-2-PyTorch-2D-CNN-train_noCV')

# Get all runs in the experiment and get their metrics (only if they are completed)
for r in list(experiment.get_runs()):

    # Get the run by using the Workspace and the run id (only if the run is completed)
    if(r.status == 'Completed'):
        run = Run.get(ws, run_id=r.id)

        # Get all metrics from the run
        metrics = run.get_metrics()
        print(metrics['Time training CNN'])
        # print(metric_name, ":", metrics[metric_name])
        """
        Available metrics:
        - Patients used for training
        - Patients used for testing
        - Batch dimensions
        - Number of epochs
        - Batch size
        - Patch size
        - Number of K folds
        - Learning rates
        - OACC
        - ACC
        - SEN
        - SPE
        - PRECISSION
        - CONFUSION MATRIX
        - Patients used to classify
        - Predicted GT classification map
        - Predicted and true GT classification maps
        - Model loss and accuracy by epoch
        - Time loading arguments
        - Time preparing train data
        - Time training CNN
        - Time preparing test data
        - Time predicting GT test image
        - Time generating classification maps
        """

    